In [ ]:
"""
generate_drive_excel_oauth.py

Requirements:
    pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib pandas openpyxl

Place your OAuth credentials file from Google Cloud Console (Desktop App) as:
    credentials.json

Run:
    python generate_drive_excel_oauth.py

Notes:
- The first run will open a browser to let you sign in and grant Drive permissions.
- The script only lists files directly inside the provided folder ID (non-recursive).
- If you want non-image files included, remove or change the mimeType check.
"""

import os
import pickle
import pandas as pd
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# -------- USER CONFIG ----------
# Replace with your folder id (from the folder link)
FOLDER_ID = "1FrbwVzp0HvnPeDLtVIpBb-ngFtgsn5uz"

# Output file
OUTPUT_XLSX = "images_metadata.xlsx"

# Numeric columns you want
NUMERIC_COLS = ["Num1", "Num2", "Num3"]

# OAuth scopes
SCOPES = [
    "https://www.googleapis.com/auth/drive.metadata.readonly",
    "https://www.googleapis.com/auth/drive.readonly"
]
# ------------------------------

def get_drive_service():
    creds = None
    token_path = "token.pickle"
    creds_path = "credentials.json"

    if os.path.exists(token_path):
        with open(token_path, "rb") as f:
            creds = pickle.load(f)

    # If there are no (valid) credentials available, do the OAuth flow.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            if not os.path.exists(creds_path):
                raise FileNotFoundError(
                    "credentials.json not found. Download OAuth desktop credentials from\n"
                    "https://console.developers.google.com/apis/credentials and save as credentials.json"
                )
            flow = InstalledAppFlow.from_client_secrets_file(creds_path, SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open(token_path, "wb") as f:
            pickle.dump(creds, f)

    service = build("drive", "v3", credentials=creds, cache_discovery=False)
    return service

def list_files_in_folder(service, folder_id, only_images=True):
    """
    Lists files directly inside the specified folder (non-recursive).
    only_images: when True, filters mimeType starting with 'image/'.
    Returns list of dicts: {id, name, mimeType}
    """
    q = f"'{folder_id}' in parents and trashed = false"
    if only_images:
        q += " and mimeType contains 'image/'"

    files = []
    page_token = None
    while True:
        try:
            resp = service.files().list(
                q=q,
                spaces="drive",
                fields="nextPageToken, files(id, name, mimeType)",
                pageToken=page_token,
                pageSize=1000
            ).execute()
        except Exception as e:
            raise RuntimeError(f"Error listing files from Drive: {e}")

        items = resp.get("files", [])
        files.extend(items)
        page_token = resp.get("nextPageToken", None)
        if not page_token:
            break
    return files

def build_links(file_id):
    """
    Build typical view and direct-download links.
    Note: access still requires adequate file sharing / the OAuth user permissions.
    """
    view_link = f"https://drive.google.com/file/d/{file_id}/view?usp=sharing"
    direct_download = f"https://drive.google.com/uc?id={file_id}&export=download"
    return view_link, direct_download

def make_dataframe(files):
    rows = []
    for f in files:
        file_id = f.get("id")
        name = f.get("name")
        label = os.path.splitext(name)[0]
        view_link, direct_download = build_links(file_id)
        row = {"Image Label": label, "Google Drive View Link": view_link, "Direct Download Link": direct_download}
        # add numeric placeholders
        for c in NUMERIC_COLS:
            row[c] = ""
        rows.append(row)

    # column order: Image Label, numeric cols..., View Link, Direct Download
    cols = ["Image Label"] + NUMERIC_COLS + ["Google Drive View Link", "Direct Download Link"]
    df = pd.DataFrame(rows, columns=cols)
    return df

def main():
    print("Starting Drive -> Excel export (OAuth)...")
    service = get_drive_service()
    print("Authenticated. Listing files in folder:", FOLDER_ID)
    files = list_files_in_folder(service, FOLDER_ID, only_images=True)
    print(f"Found {len(files)} image files.")
    df = make_dataframe(files)
    df.to_excel(OUTPUT_XLSX, index=False, engine="openpyxl")
    print(f"Done. Wrote {len(df)} rows to: {OUTPUT_XLSX}")
    print("If links show permission errors, ensure the signed-in Google account has access to those files,")
    print("or change sharing settings for the folder/files (e.g., share with the account or make 'Anyone with link' viewer).")

if __name__ == "__main__":
    main()


Starting Drive -> Excel export (OAuth)...
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=761325297316-282tch0lfelv27jt0pdiq85v2jflj0km.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A49905%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.metadata.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.readonly&state=XwK7p0FlByC205UixY7f8lthpSPKlF&access_type=offline


In [ ]:
# Jupyter-friendly get_drive_service() using run_console()
import os
import pickle
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

SCOPES = [
    "https://www.googleapis.com/auth/drive.metadata.readonly",
    "https://www.googleapis.com/auth/drive.readonly"
]

def get_drive_service(creds_path="credentials.json", token_path="token.pickle"):
    creds = None
    # load token if present
    if os.path.exists(token_path):
        with open(token_path, "rb") as f:
            creds = pickle.load(f)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            if not os.path.exists(creds_path):
                raise FileNotFoundError("credentials.json not found in working directory.")
            flow = InstalledAppFlow.from_client_secrets_file(creds_path, SCOPES)

            # Use console flow compatible with headless servers / Jupyter
            print("Running console OAuth flow. A URL will be shown — open it in your browser, sign in,")
            print("then paste the authorization code below (it may be long).")
            creds = flow.run_console()

        # save token for later (so you don't need to re-auth every run)
        with open(token_path, "wb") as f:
            pickle.dump(creds, f)

    service = build("drive", "v3", credentials=creds, cache_discovery=False)
    return service


In [ ]:
service = get_drive_service()